# Генератор ссылок на предложения

В этом файле содержится код сбора ссылок на предложения о продаже квартир в новых жилых комплексах, информация о которых была собрана в ноутбуке `new_complexes_information_grabber`. 

Для каждого жилого комплекса находится количество предложений от застройщика и количество предложений от агентов и составляются наборы ссылок на страницы с этими объявлениями. Эти наборы затем сохраняются в файлах `dev_links.txt` и `agent_links.txt` соответственно и используются в ноутбуке `offers_grabber` для получения детальной информации о самих предложениях.

In [11]:
import numpy as np
import pandas as pd
import math
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# Основные структурные части ссылки на страницу с предложениями по определенному ЖК
base_link_strs = ["https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&from_developer=", "&newobject=" , 
                  "&offer_type=flat&p="]

In [146]:
# Загружаем информацию о новых жилых комплексах
df_all_complexes = pd.read_csv("complexes_data.csv", sep=',', index_col='Unnamed: 0')
df_all_complexes

,complex_id,complex_name
0,6953,Граффити
1,7458,UP-квартал Светлановский
2,44981,YE'S Residence (Йес Резиденс)
3,51431,"Институтский, 16"
4,53790,UP-квартал Пушкинский
...,...,...
520,6882,Пифагор
521,8551,"Шлиссельбург, Пролетарская ул., 38"
522,45592,Крылья
523,6335,ЗимаЛето


In [131]:
# Достаем списки с id и названиями
ids_array = df_new_complexes['complex_id'].values
names_array = df_new_complexes['complex_name'].values

### Получение информации о количестве страниц предложений

In [137]:
# Функция для сбора информации об общем числе предложений
# complexes_ids - массив с id жилых комплексов, предложения о квартирах в которых нужно собрать
# offer_type - 0, если агентские предложения и 1, если напрямую от застройщика 
def get_info_about_number_of_offers(complexes_ids, offer_type):
    pages_number = []
    for complex_id in complexes_ids:
        offers_link = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&from_developer=" + str(offer_type) + "&newobject=" + str(complex_id) + "&offer_type=flat"
        
        # Получаем нужную страницу
        search_page = requests.get(offers_link.format(0))
        search_page = search_page.text
        search_page = BeautifulSoup(search_page, 'lxml')

        # Внутри нужной страницы достаем информацию об общем количестве предложений
        offers_part = search_page.html.head.findAll('title')
        offers_str = re.findall(r"- (.*) объявл", str(offers_part))
        print(offers_str)
        if len(offers_str) == 0:
            pages_num = 0
        else:
            if offers_str[0] == 'база':
                break
            else:
                pages_num = int(offers_str[0].replace(' ', ''))
        pages_number.append(pages_num)
    return pages_number

In [138]:
# Получаем информацию о количестве отдельно предложений от застройщика и предложений от агентов
complexes_agent_offers_pages = get_info_about_number_of_offers(ids_array, 0)
complexes_dev_offers_pages = get_info_about_number_of_offers(ids_array, 1)

In [2]:
# Восстанавливаем информацию о количестве страниц предложений из общего числа предложений
offers_per_page = 34 # Циан публикует по 34 объявления на странице
agent_offers_pages_lst = [math.ceil(x / offers_per_page) for x in complexes_agent_offers_pages]
dev_offers_pages_lst = [math.ceil(x / offers_per_page) for x in complexes_dev_offers_pages]

### Сбор ссылок на страницы с предложениями

In [120]:
# Функция, генерирующая ссылки на страницы с предложениями
# complexes_ids - массив с id жилых комплексов, предложения о квартирах в которых нужно собрать
# pages_num_arr - массив с количеством страниц предложений для каждого ЖК 
# offer_type - 0, если агентские предложения и 1, если напрямую от застройщика 
def links_generator(complexes_ids, pages_num_arr, offer_type):
    links_pool = [] 
    
    for i in range(481, 495):
        complex_id = complexes_ids[i]
        offers_pages = pages_num_arr[i - 481]
        
        for page_id in range(offers_pages):
            links_pool.append(base_link_strs[0] + str(offer_type) + base_link_strs[1] + str(complex_id) + 
                                       base_link_strs[2] + str(page_id + 1))
    return links_pool

In [121]:
from_agent_links_pool = links_generator(ids_array, agent_offers_pages_lst, 0)
from_dev_links_pool = links_generator(ids_array, dev_offers_pages_lst, 1)

In [63]:
# Сохраняем информацию о ссылках на агентские предложения 
f_ag_links = open('agents_links.txt', 'w')

for link in from_agent_links_pool:
    f_ag_links.write(link + '\n')
f_ag_links.close()

In [ ]:
# Сохраняем информацию о ссылках на предложения от застройщика
f_dev_links = open('devs_links.txt', 'w')

for link in from_dev_links_pool:
    f_dev_links.write(link + '\n')
f_dev_links.close()